In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as Func
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot 
import os
from PIL import Image
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [2]:
training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Resize(180),
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) #These are the commonly used ImageNet values
])

testing_transform = transforms.Compose([
    transforms.Resize(180),
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]) 
])

In [3]:
path = 'Data/'
training_data = datasets.ImageFolder(str(path+'train'),transform=training_transform)
testing_data = datasets.ImageFolder(str(path+'test'),transform=testing_transform)

torch.manual_seed(13)
training_loader = DataLoader(training_data,batch_size=10,shuffle=True)
testing_loader = DataLoader(testing_data,batch_size=10)

class_names = training_data.classes

In [4]:
print(class_names)

['barrel_jellyfish', 'blue_jellyfish', 'compass_jellyfish', 'lions_mane_jellyfish', 'mauve_stinger_jellyfish', 'moon_jellyfish']


In [6]:
print('Training Data: ',len(training_data))
print('Testing Data: ',len(testing_data))

Training Data:  900
Testing Data:  40


In [7]:
class ConvolutionalNetwork(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(43*43*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,6)
        
    def forward(self, X):
        X = Func.relu(self.conv1(X))
        X = Func.max_pool2d(X,2,2)
        X = Func.relu(self.conv2(X))
        X = Func.max_pool2d(X,2,2)
        X = X.view(-1,43*43*16)
        X = Func.relu(self.fc1(X))
        X = Func.relu(self.fc2(X))
        X = self.fc3(X)
        
        return Func.log_softmax(X,dim=1)

In [8]:
torch.manual_seed(13)
CNNmodel = ConvolutionalNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNNmodel.parameters(),lr=0.00005)

In [9]:
CNNmodel

ConvolutionalNetwork(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=29584, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

In [16]:
import time
start_time = time.time()

epochs = 5


train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train,y_train) in enumerate(training_loader): #b is batches
        
        b+=1
        
        y_pred = CNNmodel(X_train)
        loss = criterion(y_pred, y_train)
        
        #Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update Parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%20 == 0:
            print(f'Epoch {i} LOSS: {loss.item()}')
            
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    # TEST SET!
    with torch.no_grad():
        for b,(X_test,y_test) in enumerate(testing_loader):
            
            y_val = CNNmodel(X_test)
            
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()
            
    loss = criterion(y_val,y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

total_time = time.time() - start_time
print(f'Total time: {total_time/60} minutes')

Epoch 0 LOSS: 1.2465614080429077
Epoch 0 LOSS: 1.0231850147247314
Epoch 0 LOSS: 1.1138523817062378
Epoch 0 LOSS: 0.8234699964523315
Epoch 1 LOSS: 0.6281180381774902
Epoch 1 LOSS: 0.9776129722595215
Epoch 1 LOSS: 1.041824221611023
Epoch 1 LOSS: 0.9389379620552063
Epoch 2 LOSS: 0.6995431780815125
Epoch 2 LOSS: 0.570053219795227
Epoch 2 LOSS: 0.778610348701477
Epoch 2 LOSS: 0.6369116306304932
Epoch 3 LOSS: 0.9067428708076477
Epoch 3 LOSS: 0.9059064984321594
Epoch 3 LOSS: 0.5900046825408936
Epoch 3 LOSS: 0.8239524960517883
Epoch 4 LOSS: 1.0311119556427002
Epoch 4 LOSS: 0.7561290264129639
Epoch 4 LOSS: 0.8839741945266724
Epoch 4 LOSS: 0.703273594379425
Total time: 0.40283884207407633 minutes
